## Chap 3 데이터 소스의 서식과 가공
* 크롤링으로 수집된 데이터를 어떻게 다루어야 하는 지 살펴본다.
* XML/JSON/CSV/엑셀 형식 데이터에 관해 살펴본다

### XML 분석
* XML 은 텍스트 데이터를 기반으로 하는 형식으로 보통 웹 API가 XML 형식을 활용
* XML 즉 eXtensible Markup Language란 특정 목적에 따라 데이터를 태그로 감싸 마크업하는 범용적인 형식으로 W3C에 의해 만들어짐
* XML은 계층구조로 표현할 수 있는 게 특징으로 어떤 데이터 아래에 서브 데이터를 추가할 수 있으며, 그러한 서브 데이터 아래에 또 다른 서브 데이터를 추가할 수 있음.
* 다음 코드는 XML의 기본적인 구조를 나타내고 있음.
```xml
<요소 속성="속성값">내용</요소>
```
* 데이터의 내용을 원하는 <요소> 태그로 감싸 마크업을 한다. 아무것이나 원하는 요소 이름을 사용하면 되고, 또한 하나의 요소에는 속성을 사용해 여러 값을 추가로 지정할 수 있음. 
```xml
<product id="S001" price="45000">SD 카드</product>
```
* 이러한 요소는 다른 요소를 그룹으로 묶는 것도 가능
```xml
<products type="전자제품">
    <product id="S001" price="45000">SD 카드</product>
    <product id="S002" price="32000">마우스</product>
</products>
```

### 파이썬으로 XML 분석하기 - 날씨 분류하기

** 기상청 전국날씨 **
<http://www.kmo.go.kr/weather/forecast/mid-term-rss3.jsp?stnID=108>

* 다음의 예제는 위의 XML 데이터를 내려 받아 현재 날씨를 분석하고 분류하는 프로그램임

In [3]:
from bs4 import BeautifulSoup 
import urllib.request as req
import os.path
url = "http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=108"
savename = "forecast.xml"
# XML 파일 내려받기
# request.urlretrieve() 함수를 사용해 처음 실행할 때 로커 파일로 데이터를 저장하고 두번째 이후 실행할 때는 저장한 데이터를 읽어 사용
if not os.path.exists(savename):
    req.urlretrieve(url, savename)
# BeautifulSoup로 분석하기 --- (※2)
xml = open(savename, "r", encoding="utf-8").read()
soup = BeautifulSoup(xml, 'html.parser')
# 각 지역 확인하기 --- (※3)
# BeautifulSoup는 모든 tag를 소문자로 변환하므로 find 등의 메소드를 쓸 때 주의해서 사용
info = {}
for location in soup.find_all("location"):
    name = location.find('city').string
    weather = location.find('wf').string
    if not (weather in info):
        info[weather] = []
    info[weather].append(name)
# 각 지역의 날씨를 구분해서 출력하기
for weather in info.keys():
    print("+", weather)
    for name in info[weather]:
        print("| - ", name)

+ 구름조금
| -  서울
| -  인천
| -  수원
| -  파주
| -  대전
| -  세종
| -  홍성
| -  청주
| -  광주
| -  목포
| -  여수
| -  전주
| -  군산
| -  부산
| -  울산
| -  창원
| -  대구
| -  안동
| -  포항
+ 구름많음
| -  춘천
| -  원주
| -  강릉
| -  제주
| -  서귀포


### JSON 분석
* JSON(JavaScript Object Notation) 도 텍스트 데이터를 기반으로 하는 가벼운 데이터 형식
* JSON은 자바스크립트에서 사용하는 객체 표기방법을 기반으로 함. JSON은 다양한 소프트웨어와 프로그래밍 언어끼리 데이터를 교환할 때 사용
* 인터넷 표준 문서 RFC4627로 표준이 만들어져 있으며 MIME 타입은 "application/json"이며 확장자는 .json임
* JSON은 구조가 단순하다는 것이 장점으로 수많은 프로그래밍 언어에서 인코딩/디코딩 표준으로 JSON을 제공하고 있음
* 파이썬 모듈에도 json이 포함되어 있으며 많은 웹 API 들이 JSON 형식으로 데이터를 제공하고 있음.
* JSON 소개 페지이: <http://json.org>

#### JSON의 구조
* JSON에서는 숫자, 문자열, 불(true|false), 배열, 객체, null이라는 6가지 종류의 데이터를 사용하고 있음.
* 각각의 표현 방법은 다음과 같음

| 자료형 | 표현방법 | 사용예 |
| --- | ---- | ---- |
| 숫자  | 숫자 | 30 |
| 문자열  | 큰 따옴표로 감싸 표현 | "str" |
| 불  | true or false | true |
| 배열  | [n1, n2, n3] | [1,2,10,500] |
| 객체  | {"key":value, "key":value,...}  | {"org":50, "com":10} |
| null  | null | null |

#### 파이썬으로 JSON 분석하기
* 파이썬에서 JSON을 다루는 것은 매우 간단한데 JSON의 배열(Array)은 파이썬의 리스트(list)와 같으며, JSON의 객체(Object)는 파이썬의 딕셔너리(dict)와 같기 때문임.
* 데이터는 <http://api.github.com/repositories> 를 이용함. 클릭해서 내용을 확인해본다.
* 다음의 예제는 이 파일을 분석해 보는 것임.

In [4]:
import urllib.request as req
import os.path, random
import json
# JSON 데이터 내려받기 --- (※1)
url = "https://api.github.com/repositories"
savename = "repo.json"
if not os.path.exists(url):
    req.urlretrieve(url, savename)
# JSON 파일 분석하기 --- (※2)
# json.load() 함수의 매개변수에는 open() 함수의 리턴값이 파일 포인터를 지정. 그리고 JSON 형식의 문자열 지정
items = json.load(open(savename, "r", encoding="utf-8"))
# 또는
# s = open(savename, "r", encoding="utf-8").read()
# items = json.loads(s)
# 출력하기 --- (※3)
for item in items:
    print(item["name"] + " - " + item["owner"]["login"])

grit - mojombo
merb-core - wycats
rubinius - rubinius
god - mojombo
jsawesome - vanpelt
jspec - wycats
exception_logger - defunkt
ambition - defunkt
restful-authentication - technoweenie
attachment_fu - technoweenie
microsis - Caged
s3 - anotherjesse
taboo - anotherjesse
foxtracs - anotherjesse
fotomatic - anotherjesse
glowstick - mojombo
starling - defunkt
merb-more - wycats
thin - macournoyer
resource_controller - jamesgolick
markaby - jamesgolick
enum_field - jamesgolick
subtlety - defunkt
zippy - defunkt
cache_fu - defunkt
phosphor - KirinDave
sinatra - bmizerany
gsa-prototype - jnewland
duplikate - technoweenie
lazy_record - jnewland
gsa-feeds - jnewland
votigoto - jnewland
mofo - defunkt
xhtmlize - jnewland
ruby-git - ruby-git
bmhsearch - ezmobius
mofo - uggedal
simply_versioned - mmower
gchart - abhay
schemr - benburkert
calais - abhay
chronic - mojombo
git-wiki - sr
signal-wiki - queso
ruby-on-rails-tmbundle - drnic
low-pro-for-jquery - danwrong
merb-core - wayneeseguin
dst - s

### JSON 형식으로 출력하기
* 파이썬에서 생성한 데이터를 JSON 형식으로 출력하는 기능도 있음. 
* JSON 형식으로 출력할 때는 json.dumps() 함수를 사용함.
* 다음의 예제는 json 형태로 만들어주고 있음

In [7]:
import json
price = {
    "date": "2018-09-20",
    "price": {
        "Apple":80, 
        "Orange":55,
        "Banana": 40
    }
}
s=json.dumps(price)
print(s)

{"date": "2018-09-20", "price": {"Apple": 80, "Orange": 55, "Banana": 40}}


### CSV, TSV 분석
* CSV(Comma Seperated valeus)는 각 필드를 쉼표로 구분함. 텍스트 에디터를 사용해 간단하게 수정가능. TSV는 tab으로 구분된 파일이고 space로 구분된 SSV 파일도 있음.

#### 파이썬에서 단순한 CSV 파일 읽기
* 간단하게 다음의 CSV 파일을 읽어 list-euckr.csv 로 저장하자. 

ID, 이름, 가격
1000, 비누, 300
1001, 장갑, 150
1002, 마스크, 230

* 다음은 이러한 csv 파일을 읽어들이는 예제임.

In [8]:
import codecs
# EUC_KR로 저장된 CSV 파일 읽기
filename = "list-euckr.csv"
csv = codecs.open(filename, "r", "euc_kr").read()
# CSV을 파이썬 리스트로 변환하기
data = []
rows = csv.split("\r\n")
for row in rows:
    if row == "": continue
    cells = row.split(",")
    data.append(cells)
# 결과 출력하기
for c in data:
    print(c[1], c[2])

이름 가격
비누 300
장갑 150
마스크 230


#### 파이썬의 csv 모듈 사용하기
* 대용량 CSV 파일을 분석하기 위해서는 csv 모듈을 이용하는 것이 편리함
* 다음은 이전의 CSV 파일을 csv 모듈을 이용해 읽어들이는 예제임

In [12]:
import csv, codecs
# CSV 파일 열기
filename = "list-euckr.csv"
# 한글을 읽어들이기 위해 euc-kr 이용
fp = codecs.open(filename, "r", "euc-kr")
# 한 줄씩 읽어 들이기, delimiter는 어떻게 구분되어 있는지 quotechar는 어떤 기호로 데이터를 감싸고 있는 지 지정 가능함.
reader = csv.reader(fp, delimiter=",", quotechar='"')
for cells in reader:
    print(cells[1], cells[2])

이름 가격
비누 300
장갑 150
마스크 230


#### CSV 파일  쓰기
* 다음은 test.csv 파일을 생성하는 예제임

In [13]:
import csv, codecs
with codecs.open("test.csv", "w", "euc_kr ") as fp:
    writer = csv.writer(fp, delimiter=",", quotechar='"')
    writer.writerow(["ID", "이름", "가격"])
    writer.writerow(["1000", "SD 카드 ", 30000])
    writer.writerow(["1001", "키보드", 21000])
    writer.writerow(["1002", "마우스", 15000])

### Pandas 사용해 보기
* 데이터 분석 라이브러리 Pandas를 이용하면 CSV 파일을 쉽게 읽고 편집하는 것이 가능함
* 엑셀 파일을 읽고 쓰기 위해 파이썬에서는 openpyxl을 제고앟고 있는데 이를 이용해서 엑셀을 읽고 쓰는 것이 가능함.

* Terminal에서 "pip install openpyxl" 설치해보자.
* 엑셀 파일을 읽어보기 위해 <http://www.index.go.kr"> 국가 지표 체계 사이트를 이용하여 엑셀을 내려받자.
* 다음은 자치단체 행정구역 및 인구현황 엑셀파일이 있는 페이지로서 여기서 엑셀을 다운로드 받아 적절한 이름으로 다시 저장하자.
* openpyxl의 경우 최신 엑셀 파일만 지원하므로 xlsx로 저장해야 한다.
* <http://www.index.go.kr/potal/main/EachDtlPageDetail.do?idx_cd=1041>
* 다음의 예제는 엑셀 파일을 읽고 2015년도의 인구가 적은 순서대로 5위까지 출력하고 있음.

In [103]:
import openpyxl 
# 엑셀 파일 열기く --- (※1)
filename = "stat_104102.xlsx"
book = openpyxl.load_workbook(filename)
# 맨 앞의 시트 추출하기 --- (※2)
sheet = book.worksheets[0]
# 시트의 각 행을 순서대로 추출하기 --- (※3)
data = []

for row in sheet.rows:
    data.append([
        row[0].value,
        row[10].value
    ])
# 필요없는 줄(헤더, 연도, 계) 제거하기, 마지막부분 제거하기
data1=data[3:21]
# 데이터를 인구 순서로 정렬합니다.
data1 = sorted(data1, key=lambda x:x[1])
# 하위 5위를 출력합니다.
for i, a in enumerate(data1):
    if (i >= 5): break
    print(i+1, a[0], int(a[1]))

1 세종 280
2 제주 657
3 울산 1165
4 광주 1463
5 대전 1502


#### 파이썬으로 엑셀파일 쓰기
* 엑셀 파일로 저장해보기 

In [111]:
import openpyxl 
# 엑셀 파일 열기 --- (※1)
filename = "stat_104102.xlsx"
book = openpyxl.load_workbook(filename)
# 활성화된 시트 추출하기 --- (※2)
sheet = book.active
# 서울을 제외한 인구를 구해서 쓰기 --- (※3)
for i in range(0, 9):
    total = int(sheet[str(chr(i + 66)) + "4"].value)
    seoul = int(sheet[str(chr(i + 66)) + "5"].value)
    output = total - seoul
    print("서울 제외 인구 =", output)
    # 쓰기 --- (※4)
    sheet[str(chr(i + 66)) + "21"] = output
    cell = sheet[str(chr(i + 66)) + "21"]
    # 폰트와 색상 변경해보기 --- (※5)
    cell.font = openpyxl.styles.Font(size=14,color="FF0000")
    cell.number_format = cell.number_format
# 엑셀 파일 저장하기 --- (※6)
filename = "population.xlsx"
book.save(filename)
print("ok")

서울 제외 인구 = 39339
서울 제외 인구 = 39565
서울 제외 인구 = 40203
서울 제외 인구 = 40484
서울 제외 인구 = 40753
서울 제외 인구 = 40997
서울 제외 인구 = 41225
서울 제외 인구 = 41507
서울 제외 인구 = 41766
ok


### Pandas를 이용해 엑셀 읽고 쓰기
* pandas, xlrd를 이용해 좀 더 쉽게 엑셀을 읽고 쓰는 것이 가능함

In [123]:
# import pandas as pd
# 엑셀 파일 열기 --- (※1)
filename = "stat_104102-1.xlsx" # 파일 이름
book = pd.read_excel(filename) 
# 2017년 인구로 정렬 --- (※2)
book = book.sort_values(by=2017, ascending=False)
print(book)

     2008   2009   2010   2011   2012   2013   2014   2015   2016   2017
계   49540  49773  50515  50734  50948  51141  51328  51529  51696  51778
경기  11292  11460  11787  11937  12093  12235  12358  12522  12716  12873
서울  10201  10208  10312  10250  10195  10144  10103  10022   9930   9857
부산   3565   3543   3568   3551   3538   3528   3519   3513   3498   3470
경남   3225   3250   3291   3309   3319   3333   3350   3364   3373   3380
인천   2693   2710   2758   2801   2844   2880   2903   2925   2943   2948
경북   2674   2669   2690   2699   2698   2699   2701   2702   2700   2691
대구   2493   2489   2512   2508   2506   2502   2493   2487   2484   2475
충남   2019   2037   2075   2101   2029   2048   2062   2077   2096   2116
전남   1919   1913   1918   1914   1910   1907   1906   1908   1903   1896
전북   1856   1854   1869   1874   1873   1873   1872   1869   1864   1854
충북   1520   1527   1549   1563   1566   1573   1579   1583   1591   1594
강원   1509   1512   1530   1536   1539   1542   1544

### Pandas 사용법
* 다음의 URL에서 pandas를 공부해보자
<https://dandyrilla.github.io/2017-08-12/pandas-10min/>